# 1. 필요 라이브러리 가져오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as numpy
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import warnings
# import koreanize_matplotlib
warnings.filterwarnings('ignore')
%matplotlib inline

# 2, 데이터셋 확인

In [4]:
df_train = pd.read_csv('/content/drive/MyDrive/data/감귤/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/data/감귤/test.csv')

In [6]:
df_train.head()

,ID,착과량(int),수고(m),수관폭1(min),수관폭2(max),수관폭평균,2022-09-01 새순,2022-09-02 새순,2022-09-03 새순,2022-09-04 새순,...,2022-11-19 엽록소,2022-11-20 엽록소,2022-11-21 엽록소,2022-11-22 엽록소,2022-11-23 엽록소,2022-11-24 엽록소,2022-11-25 엽록소,2022-11-26 엽록소,2022-11-27 엽록소,2022-11-28 엽록소
0,TRAIN_0000,692,275.0,287.0,292.0,289.5,2.8,2.8,2.7,2.7,...,70.978249,70.876794,70.705253,70.559603,70.427356,70.340491,70.293830,70.262422,70.169841,70.043251
1,TRAIN_0001,534,293.0,284.0,336.0,310.0,3.3,3.3,3.3,3.2,...,71.535483,71.382303,71.253604,71.092665,70.955608,70.796630,70.597550,70.565088,70.560502,70.427600
2,TRAIN_0002,634,300.0,392.0,450.0,421.0,3.0,2.9,2.9,2.9,...,71.279804,71.199570,71.144020,71.026740,70.920038,70.876723,70.710129,70.595971,70.418203,70.399578
3,TRAIN_0003,639,289.0,368.0,379.0,373.5,3.1,3.0,3.0,3.0,...,69.934615,69.884124,69.845683,69.794682,69.779813,69.614644,69.455404,69.281590,69.238689,69.133970
4,TRAIN_0004,496,306.0,353.0,358.0,355.5,3.7,3.6,3.6,3.6,...,68.313016,68.285364,68.209860,68.209458,68.040083,67.859963,67.775556,67.701456,67.504244,67.410093


In [7]:
df_test.head()

,ID,수고(m),수관폭1(min),수관폭2(max),수관폭평균,2022-09-01 새순,2022-09-02 새순,2022-09-03 새순,2022-09-04 새순,2022-09-05 새순,...,2022-11-19 엽록소,2022-11-20 엽록소,2022-11-21 엽록소,2022-11-22 엽록소,2022-11-23 엽록소,2022-11-24 엽록소,2022-11-25 엽록소,2022-11-26 엽록소,2022-11-27 엽록소,2022-11-28 엽록소
0,TEST_0000,231.0,219.0,302.0,260.5,4.3,4.2,4.2,4.1,4.1,...,72.091497,71.906176,71.886799,71.858720,71.725540,71.531618,71.476935,71.321237,71.184295,71.000223
1,TEST_0001,206.0,324.0,346.0,335.0,2.5,2.5,2.5,2.5,2.5,...,68.423776,68.353200,68.346777,68.188159,67.995233,67.879958,67.744820,67.673522,67.539973,67.447849
2,TEST_0002,282.0,373.0,404.0,388.5,4.7,4.7,4.6,4.6,4.5,...,74.435783,74.247468,74.142664,74.070401,73.892408,73.796143,73.615838,73.440136,73.387051,73.228319
3,TEST_0003,234.0,337.0,357.0,347.0,3.6,3.6,3.6,3.6,3.5,...,73.570735,73.420243,73.313538,73.149298,73.109599,72.924511,72.847411,72.838263,72.672156,72.538171
4,TEST_0004,222.0,329.0,347.0,338.0,2.7,2.7,2.7,2.7,2.6,...,72.216758,72.163153,72.052952,71.905982,71.766141,71.746813,71.668847,71.501043,71.419715,71.328224


In [8]:
df_train.columns

Index(['ID', '착과량(int)', '수고(m)', '수관폭1(min)', '수관폭2(max)', '수관폭평균',
       '2022-09-01 새순', '2022-09-02 새순', '2022-09-03 새순', '2022-09-04 새순',
       ...
       '2022-11-19 엽록소', '2022-11-20 엽록소', '2022-11-21 엽록소', '2022-11-22 엽록소',
       '2022-11-23 엽록소', '2022-11-24 엽록소', '2022-11-25 엽록소', '2022-11-26 엽록소',
       '2022-11-27 엽록소', '2022-11-28 엽록소'],
      dtype='object', length=184)

## 2.1 Null data check

In [9]:
for col in df_train.columns:
    msg = '변수명: {:>10}\t NaN 값의 확률: {:.2f}%'.format(col, 100 * (df_train[col].isnull().sum() / df_train[col].shape[0]))
    print(msg)

변수명:         ID	 NaN 값의 확률: 0.00%
변수명:   착과량(int)	 NaN 값의 확률: 0.00%
변수명:      수고(m)	 NaN 값의 확률: 0.00%
변수명:  수관폭1(min)	 NaN 값의 확률: 0.00%
변수명:  수관폭2(max)	 NaN 값의 확률: 0.00%
변수명:      수관폭평균	 NaN 값의 확률: 0.00%
변수명: 2022-09-01 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-02 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-03 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-04 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-05 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-06 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-07 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-08 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-09 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-10 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-11 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-12 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-13 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-14 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-15 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-16 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-17 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-18 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-19 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-20 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-21 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-22 새순	

In [10]:
for col in df_test.columns:
    msg = '변수명: {:>10}\t NaN 값의 확률: {:.2f}%'.format(col, 100 * (df_test[col].isnull().sum() / df_test[col].shape[0]))
    print(msg)

변수명:         ID	 NaN 값의 확률: 0.00%
변수명:      수고(m)	 NaN 값의 확률: 0.00%
변수명:  수관폭1(min)	 NaN 값의 확률: 0.00%
변수명:  수관폭2(max)	 NaN 값의 확률: 0.00%
변수명:      수관폭평균	 NaN 값의 확률: 0.00%
변수명: 2022-09-01 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-02 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-03 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-04 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-05 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-06 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-07 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-08 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-09 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-10 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-11 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-12 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-13 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-14 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-15 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-16 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-17 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-18 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-19 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-20 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-21 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-22 새순	 NaN 값의 확률: 0.00%
변수명: 2022-09-23 

* train, test 데이터 모두 null data가 없다.